In [156]:
import numpy as np
import pandas as pd
import pickle
import yaml
import sys
from audit import exp_one_acc, exp_all_acc, exp_worst_eps, exp_estimated_epsilon, exp_all_avg_acc, exp_all_group_avg_acc
from visual import *
from util import *
import json

In [172]:
# configs = "configs/raceface/raceface_regular.yaml"
# configs = "configs/mnist/mnist_regular.yaml"
configs = "configs/tabular/adult/adult_regular.yaml"
with open(configs, "rb") as f:
    configs = yaml.load(f, Loader=yaml.Loader)

dataset_name = configs["data"]["dataset"]
num_group = configs["train"]["num_groups"]
model_name = "LR"
epsilon = 0
method = "regular"

In [173]:
subdata_dir = f'{configs["run"]["log_dir"]}{dataset_name}.pkl'
# exp_dir = f'{configs['run']['log_dir']}{model_name}/{method}/data/data_{data_idx}/eps{epsilon}'
exp_dir = f'{configs["run"]["log_dir"]}{model_name}/{method}/eps{epsilon}'
log_dir = exp_dir
directories = {
    "log_dir": log_dir,
    "report_dir": f"{log_dir}/report",
    "signal_dir": f"{log_dir}/signals",
}

path = subdata_dir
if num_group == 5 or num_group == 2:
    path = f"data/tabular/{dataset_name}.pkl"
with open(path, "rb") as file:
    dataset = pickle.load(file)

path = directories["report_dir"]
memberships = np.load(f"{path}/memberships.npy")
loss_scores = np.load(f"{path}/loss_scores.npy")


all_acc = exp_all_acc(f"{directories['report_dir']}/loss", loss_scores, memberships, dataset, num_group)
all_adv_acc = [[2*value-1 for value in group] for group in all_acc]

In [174]:
len(dataset)

45222

In [159]:
## 测试精度
path = f"{configs["train"]["log_dir"]}{model_name}/{method}/eps{epsilon}"
with open(f"{path}/models/models_metadata.json", "r") as file:
    data = json.load(file)

test_accs = [entry["test_acc"] for entry in data.values()]
# 计算平均值和标准差
mean_test_acc = np.mean(test_accs) * 100
std_test_acc = np.std(test_accs)*100
print(f"Mean test accuracy: {mean_test_acc:.2f}")
print(f"Standard deviation: {std_test_acc:.4f}")

Mean test accuracy: 81.95
Standard deviation: 0.0521


In [160]:
group_mean = [np.mean([value for value in group]) for group in all_acc]
print(group_mean)
# v = (max(group_mean) - min(group_mean)) *100
# print(f"{v:.3f}")

[0.5380736877523553, 0.5365177782685512]


In [161]:
group_adv_mean = [np.mean([value for value in group]) for group in all_adv_acc]
print(group_adv_mean)
v = (max(group_adv_mean) - min(group_adv_mean)) *100
print(f"{v:.3f}")

[0.07614737550471064, 0.07303555653710246]
0.311


In [162]:
# worst eps
v = exp_worst_eps(loss_scores, memberships)
print(f"{v:.3f}")

3.974


In [163]:
# for i in range(num_group):
#     print(max(all_acc[i]), end=' ')

# group_mean = [np.mean([value for value in group]) for group in all_acc]
# print(group_mean)

In [164]:
# """
# 可以用在跑多次pa-alooa 有多个group_adv_mean
# """
# # Kruskal-Wallis 检验
# from scipy.stats import kruskal

# stat, p = kruskal(*all_adv_acc)
# print(f"统计量: {stat}, p值: {p}")

# # Mann-Whitney U 检验
# from scipy.stats import mannwhitneyu

# stat, p = mannwhitneyu(all_adv_acc[1], all_adv_acc[5], alternative='two-sided')
# print(f"统计量: {stat}, p值: {p}")

## avearge mia

In [165]:
# # configs = "configs/raceface/raceface_regular.yaml"
# configs = "configs/mnist/mnist_regular.yaml"
# # configs = "configs/tabular/law/law_regular.yaml"
# with open(configs, "rb") as f:
#     configs = yaml.load(f, Loader=yaml.Loader)

# dataset_name = configs["data"]["dataset"]
# num_group = configs["train"]["num_groups"]
# model_name = "CNN"
# epsilon = 10
# method = "dpsgd"

In [166]:
# subdata_dir = f'{configs["run"]["log_dir"]}{dataset_name}.pkl'
# # exp_dir = f'{configs['run']['log_dir']}{model_name}/{method}/data/data_{data_idx}/eps{epsilon}'
# exp_dir = f'{configs["run"]["log_dir"]}{model_name}/{method}/eps{epsilon}'
# log_dir = exp_dir
# directories = {
#     "log_dir": log_dir,
#     "report_dir": f"{log_dir}/report",
#     "signal_dir": f"{log_dir}/signals",
# }

# path = subdata_dir
# if num_group == 5 or num_group == 2:
#     path = f"data/tabular/{dataset_name}.pkl"
# with open(path, "rb") as file:
#     dataset = pickle.load(file)

# path = directories["report_dir"]
# memberships = np.load(f"{path}/memberships.npy")
# loss_scores = np.load(f"{path}/loss_scores.npy")

# # all_avg_acc = exp_all_avg_acc(f"{directories['report_dir']}/loss", loss_scores, memberships, dataset, num_group)
# all_avg_acc = exp_all_group_avg_acc(f"{directories['report_dir']}/loss", loss_scores, memberships, dataset, num_group)
# all_adv_acc = [[2*value-1 for value in group] for group in all_avg_acc]

In [167]:
# group_mean = [np.mean([value for value in group]) for group in all_avg_acc]
# print(group_mean)
# # v = (max(group_mean) - min(group_mean)) *100
# # print(f"{v:.3f}")

In [168]:
# group_adv_mean = [np.mean([value for value in group]) for group in all_adv_acc]
# print(group_adv_mean)
# v = (max(group_adv_mean) - min(group_adv_mean)) *100
# print(f"{v:.3f}")